# Visualización de ajuste de modelo lineal a un conjunto de datos de precios de casas.

Primeramente vamos a leer los datos y crear un gráfico de dispersión para observarlos

In [ ]:
import requests
import numpy as np
import matplotlib.pyplot as plt

url = 'https://raw.githubusercontent.com/noe-tec/machine-learning/main/datasets/ejemplo_data_precio_casas.csv'
response = requests.get(url)

if response.status_code == 200:
    lines = response.text.strip().splitlines()
    data = [list(map(float, line.split(','))) for line in lines]
    for row in data[:5]:  # mostramos 5 primeras lineas
        print(row)
    data = np.array(data)
    print(data.shape)
else:
    print(f'Error al descargar el archivo: {response.status_code}')

x_train = data[:,  0] 
y_train = data[:, -1] / 1e6

plt.scatter(x_train, y_train)
plt.xlabel("Tamaño [m^2]")
plt.ylabel("Precio [millones de pesos]")
plt.ylim((0,9))
plt.grid()


Ahora vamos a crear un gráfico interactivo que nos permita seleccionar valores de los parámetros "w" y "b" y ver la línea resultante. El gráfico también nos muestra un gráfico de contornos de la función costo J(w,b) y el valor actual del costo dados los parámetros.

In [ ]:
from ipywidgets import interact, FloatSlider


# --- Funciones auxiliares ---
def predict(x, w, b):
    return w * x + b

def compute_cost(x, y, w, b):
    preds = predict(x, w, b)
    return np.mean((preds - y)**2)

# --- Creamos una rejilla con combinaciones de w y b y calculamos costo ---
w_range = np.linspace(-0.1, 0.1, 100)
b_range = np.linspace(-5, 5, 100)
W, B = np.meshgrid(w_range, b_range)
Z = np.zeros_like(W)

for i in range(W.shape[0]):
    for j in range(W.shape[1]):
        Z[i, j] = compute_cost(x_train, y_train, W[i, j], B[i, j])

# --- Visualización interactiva completa ---
def plot_interactive(w=50000, b=500000):
    fig, axs = plt.subplots(1, 2, figsize=(16, 6))

    # Subplot izquierdo: regresión
    axs[0].scatter(x_train, y_train, label='Datos reales')
    x_line = np.linspace(x_train.min(), x_train.max(), 10)
    y_line = predict(x_line, w, b)
    axs[0].plot(x_line, y_line, color='red', label=f'y = {w:.3f}x + {b:.3f}')
    axs[0].set_title(f'Regresión lineal\nCosto (MSE): {compute_cost(x_train, y_train, w, b):,.0f}')
    axs[0].set_xlabel("Tamaño [m²]")
    axs[0].set_ylabel("Precio [mxn]")
    axs[0].set_ylim((-10, 10))
    axs[0].set_xlim(( 0, x_train.max()))
    axs[0].grid(True)
    axs[0].legend()

    # Subplot derecho: contornos de la función de costo
    levels = np.logspace(0, 2.5, 10)  # Usamos niveles logarítmicos para mejor visualización
    CS = axs[1].contour(W, B, Z, levels=levels, cmap='viridis')
    #CS = axs[1].contour(W, B, Z, levels=50, cmap='viridis')
    axs[1].set_title("Contornos de la función de costo")
    axs[1].set_xlabel("w (pendiente)")
    axs[1].set_ylabel("b (intersección)")
    axs[1].plot(w, b, 'ro', label='Posición actual')
    axs[1].legend()
    axs[1].grid(True)

    plt.tight_layout()
    plt.show()

# --- Sliders interactivas ---
interact(plot_interactive,
         w=FloatSlider(min=-0.1, max=0.1, step=0.001, value=0, description='w (pendiente)'),
         b=FloatSlider(min=-3, max=3, step=0.01, value=0, description='b (intercepto)'))

Actividad: Modifica los valores de "w" y "b" usando los sliders. Observa como la linea resutante varía y el costo cambia. ¿Que valores de w y b producen un buen ajuste?